In [41]:
DATA_DIR = 'input'

In [42]:
import os
from scipy.io import loadmat

# Load the camera calibration matrix data
calib = loadmat(os.path.join(DATA_DIR, 'dino_Ps.mat'))


In [43]:
import numpy as np

matrices = calib['P'][0]

In [44]:
import cv2 as cv

# Read the images in the input folder
IMAGES_DIR = os.path.join(DATA_DIR, 'images')

images = []
for file_name in os.listdir(IMAGES_DIR):
    image = cv.imread(os.path.join(IMAGES_DIR, file_name))
    images.append(image)

In [45]:
# Apply triangulation to the points

#images_3d = cv.triangulatePoints(np.array(images), matrices)

In [46]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index(index: int) -> PIL.Image:
    rgb_image = cv.cvtColor(images[index], cv.COLOR_BGR2RGB)
    return PIL.Image.fromarray(rgb_image)

wg.interact(
    show_image_by_index,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [47]:
# Crop Image black borders
images = [img[5:, :-30] for img in images]

In [48]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index(index: int) -> PIL.Image:
    rgb_image = cv.cvtColor(images[index], cv.COLOR_BGR2RGB)
    return PIL.Image.fromarray(rgb_image)

wg.interact(
    show_image_by_index,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [49]:
images_blur = [cv.GaussianBlur(image, (21, 21), 0) for image in images]

In [50]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(images_blur[index], cv.COLOR_BGR2RGB))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images_blur)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [51]:
images_hsv = [cv.cvtColor(image, cv.COLOR_BGR2LAB) for image in images_blur]
images_mask = [
    cv.threshold(img[..., 2], 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    for img in images_hsv]

In [52]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(images_mask[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [53]:
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
closing = [cv.morphologyEx(img, cv.MORPH_CLOSE, kernel) for img in images_mask]

In [54]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(closing[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [55]:
segmented_images = [cv.bitwise_and(img, img, mask=mask) for img, mask in zip(images, closing)]

In [56]:
#segmented_images

In [57]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(segmented_images[index], cv.COLOR_BGR2RGB))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [58]:
'''import open3d as o3d

def carve_sides():
	print("creating voxel space...")
	voxel_grid = o3d.geometry.VoxelGrid.create_dense(
     	origin=[-1,-1,-1],
     	color=[1,0,1],
     	voxel_size=0.00325,
     	width=2,
     	height=2,
     	depth=2)
	print("created voxel space!")
	
	file_num = 1
	for img, params in zip(segmented_images, matrices):
		voxel_grid.carve_silhouette(img, params)
		print(file_num)
		file_num = file_num + 1
		
	o3d.visualization.draw_geometries([voxel_grid])
#carve_sides('./temple/','./temple/temple_par.txt', './temple/temple_ang.txt')''';

In [59]:
matrices[0]

array([[ 3.99235688e+00,  3.94176810e+01, -7.63289880e-01,
         3.95917551e+00],
       [-1.44302310e+01, -9.41441580e-01, -2.74509701e+01,
        -1.44294334e+01],
       [ 1.22492404e-02, -1.45746038e-04, -5.69307087e-04,
         1.22493587e-02]])

In [60]:
'''
import matplotlib.pyplot as plt

# Define the 3D point in homogeneous coordinate in order to multiply with the PPM 
_3d_point_homogeneous = np.array([[0.], [0.], [0.], [1.]])

# Multiply by the PPM obtaining the pixel in homogeneous coordinate
_2d_point_homogeneous = np.matmul(matrices[0], _3d_point_homogeneous)

# Divide by the third value to get the pixel coordinate
_2d_point = _2d_point_homogeneous / _2d_point_homogeneous[-1, 0]

print("2D point using PPM (Coordinates referred to the undistorted image): ", ( int(round(_2d_point[0,0])), int(round(_2d_point[1,0])) ))
# Draw the point
img2 = np.copy(segmented_images[0])
cv.circle(img2, (_2d_point[0], _2d_point[1]), 100, (255, 0, 0), 20)

# Crop away the black region obtained by undistorsion
#img2 = img2[y:y+h_2, x:x+w_2]

plt.figure(figsize=(10, 10))
plt.imshow(img2)
plt.show()''';

In [61]:
imgH, imgW, __ = segmented_images[0].shape

https://github.com/zinsmatt/SpaceCarving/blob/master/space_carving.py

https://github.com/zinsmatt/SpaceCarving/issues/7

In [473]:
#%%
# create voxel grid
s = 50 #120
x, y, z = np.mgrid[:s, :s, :s]
pts = np.vstack((x.flatten(), y.flatten(), z.flatten())).astype(float)
pts = pts.T
nb_points_init = pts.shape[0]
xmax, ymax, zmax = np.max(pts, axis=0)
pts[:, 0] /= xmax
pts[:, 1] /= ymax
pts[:, 2] /= zmax
center = pts.mean(axis=0)
pts -= center
pts /= 5
pts[:, 2] -= 0.62
pts = np.vstack((pts.T, np.ones((1, nb_points_init))))

In [474]:
pts.shape

(4, 125000)

In [204]:
'''print(np.unique(pts[0]))
print(np.unique(pts[1]))
print(np.unique(pts[2]))''';

[-0.1        -0.09591837 -0.09183673 -0.0877551  -0.08367347 -0.07959184
 -0.0755102  -0.07142857 -0.06734694 -0.06326531 -0.05918367 -0.05510204
 -0.05102041 -0.04693878 -0.04285714 -0.03877551 -0.03469388 -0.03061224
 -0.02653061 -0.02244898 -0.01836735 -0.01428571 -0.01020408 -0.00612245
 -0.00204082  0.00204082  0.00612245  0.01020408  0.01428571  0.01836735
  0.02244898  0.02653061  0.03061224  0.03469388  0.03877551  0.04285714
  0.04693878  0.05102041  0.05510204  0.05918367  0.06326531  0.06734694
  0.07142857  0.0755102   0.07959184  0.08367347  0.0877551   0.09183673
  0.09591837  0.1       ]
[-0.1        -0.09591837 -0.09183673 -0.0877551  -0.08367347 -0.07959184
 -0.0755102  -0.07142857 -0.06734694 -0.06326531 -0.05918367 -0.05510204
 -0.05102041 -0.04693878 -0.04285714 -0.03877551 -0.03469388 -0.03061224
 -0.02653061 -0.02244898 -0.01836735 -0.01428571 -0.01020408 -0.00612245
 -0.00204082  0.00204082  0.00612245  0.01020408  0.01428571  0.01836735
  0.02244898  0.02653061 

In [218]:
'''np.mgrid[:s, :s, :s]''';

array([[[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],

        [[ 1,  1,  1, ...,  1,  1,  1],
         [ 1,  1,  1, ...,  1,  1,  1],
         [ 1,  1,  1, ...,  1,  1,  1],
         ...,
         [ 1,  1,  1, ...,  1,  1,  1],
         [ 1,  1,  1, ...,  1,  1,  1],
         [ 1,  1,  1, ...,  1,  1,  1]],

        [[ 2,  2,  2, ...,  2,  2,  2],
         [ 2,  2,  2, ...,  2,  2,  2],
         [ 2,  2,  2, ...,  2,  2,  2],
         ...,
         [ 2,  2,  2, ...,  2,  2,  2],
         [ 2,  2,  2, ...,  2,  2,  2],
         [ 2,  2,  2, ...,  2,  2,  2]],

        ...,

        [[47, 47, 47, ..., 47, 47, 47],
         [47, 47, 47, ..., 47, 47, 47],
         [47, 47, 47, ..., 47, 47, 47],
         ...,
         [47, 47, 47, ..., 47, 47, 47],
         [47, 47, 47, ..., 47, 47, 47],
    

In [535]:
#%%
# create voxel grid
s = 120
#x, y, z = np.meshgrid(np.linspace(-.1, .1, s), np.linspace(-.1, .1, s), np.linspace(-.8, -.5, s))
x, y, z = np.meshgrid(np.arange(s), np.arange(s), np.arange(s))
x, y, z = x.flatten(), y.flatten(), z.flatten()
x = -.05 + (x - x.min()) * (.05 - (-.05)) / (x.max() - x.min())
y = -.1 + (y - y.min()) * (.04 - (-.1)) / (y.max() - y.min())
z = -.75 + (z - z.min()) * (-.5 - (-.75)) / (z.max() - z.min())
#print(x.shape, y.shape, z.shape)
pts = np.vstack((x, y, z, np.ones(x.shape))).astype(float)

In [536]:
pts.shape

(4, 1728000)

In [537]:
'''print(np.unique(pts[0]))
print(np.unique(pts[1]))
print(np.unique(pts[2]))''';

In [538]:
'''res = 50

xAxis = np.linspace(-180, 90, res)
yAxis = np.linspace(-80, 70, res)
zAxis = np.linspace(-20, 460, res)

pts = np.vstack((xAxis.flatten(), yAxis.flatten(), zAxis.flatten())).astype(float)
pts = pts.T
nb_points_init = pts.shape[0]

pts = np.vstack((pts.T, np.ones((1, nb_points_init))))''';

In [539]:
#pts

In [540]:
#closing[0].shape

In [541]:
'''prova = (matrices[0] @ pts)
prova /= prova[-1, :]''';

In [542]:
#prova = np.round(prova).astype(int)

In [543]:
#prova

In [544]:
img_with_voxels = []

for P, im in zip(matrices, closing):
    im = im.copy()
    uvs = P @ pts
    uvs /= uvs[-1, :]
    uvs = np.round(uvs).astype(int)
    im = cv.cvtColor(im, cv.COLOR_GRAY2BGRA)
    for uv in uvs.T:  # Iterate by column
        im = cv.circle(im, (uv[0], uv[1]), radius=1, color=(0, 0, 255, 50), thickness=-1)
    img_with_voxels.append(im)

In [545]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(img_with_voxels[index], cv.COLOR_BGR2RGBA))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(img_with_voxels)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=35), Output()), _dom_classes=('widget-intera…

In [546]:
a = P @ pts
a /= a[-1, :]
np.unique(a)

array([-116.99115155, -116.6973345 , -116.40340866, ...,  607.08516581,
        607.56171242,  608.03845555])

In [547]:
filled = []
for P, im in zip(matrices, closing):
    # Compute the pixel coordinates of the points
    uvs = P @ pts
    # Divide by the third value to get the pixel coordinate
    uvs /= uvs[-1, :]
    # Convert the points to integer
    uvs = np.round(uvs).astype(int)
    # 
    x_good = np.logical_and(uvs[0, :] >= 0, uvs[0, :] < imgW)
    y_good = np.logical_and(uvs[1, :] >= 0, uvs[1, :] < imgH)
    good = np.logical_and(x_good, y_good)
    indices = np.where(good)[0]
    fill = np.zeros(uvs.shape[1])
    sub_uvs = uvs[:-1, indices]
    res = im[sub_uvs[1, :], sub_uvs[0, :]]
    fill[indices] = res 

    filled.append(fill)
filled = np.vstack(filled)

# the occupancy is computed as the number of camera in which the point "seems" not empty
occupancy = np.sum(filled, axis=0)

# Select occupied voxels
pts = pts.T
good_points = pts[occupancy > 4, :]

In [590]:
good_points = pts[occupancy > 8_000, :]
good_points.shape

(86732, 4)

In [591]:
img_with_3D = []

for P, im in zip(matrices, closing):
    im = np.zeros_like(im)
    uvs = P @ good_points.T
    uvs /= uvs[-1, :]
    uvs = np.round(uvs).astype(int)
    im = cv.cvtColor(im, cv.COLOR_GRAY2BGRA)
    for uv in uvs.T:  # Iterate by column
        im = cv.circle(im, (uv[0], uv[1]), radius=1, color=(0, 0, 255, 30), thickness=-1)
    img_with_3D.append(im)

In [592]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(img_with_3D[index], cv.COLOR_BGR2RGBA))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(img_with_3D)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=35), Output()), _dom_classes=('widget-intera…

In [553]:
#pts.shape

(1728000, 4)

In [552]:
#good_points.shape

(1669801, 4)

In [548]:
#%% save point cloud with occupancy scalar 
filename = "shape.txt"
with open(filename, "w") as fout:
    fout.write("x,y,z,occ\n")
    for occ, p in zip(occupancy, pts[:, :3]):
        fout.write(",".join(p.astype(str)) + "," + str(occ) + "\n")

#%% save as rectilinear grid (this enables paraview to display its iso-volume as a mesh)
import vtk

xCoords = vtk.vtkFloatArray()
x = pts[::s*s, 0]
y = pts[:s*s:s, 1]
z = pts[:s, 2]
for i in x:
    xCoords.InsertNextValue(i)
yCoords = vtk.vtkFloatArray()
for i in y:
    yCoords.InsertNextValue(i)
zCoords = vtk.vtkFloatArray()
for i in z:
    zCoords.InsertNextValue(i)
values = vtk.vtkFloatArray()
for i in occupancy:
    values.InsertNextValue(i)
rgrid = vtk.vtkRectilinearGrid()
rgrid.SetDimensions(len(x), len(y), len(z))
rgrid.SetXCoordinates(xCoords)
rgrid.SetYCoordinates(yCoords)
rgrid.SetZCoordinates(zCoords)
rgrid.GetPointData().SetScalars(values)

writer = vtk.vtkXMLRectilinearGridWriter()
writer.SetFileName("shape.vtr")
writer.SetInputData(rgrid)
writer.Write()

1

In [7]:
from typing import List
from sklearn.mixture import GaussianMixture

def get_gaussian_mixture_segmented_image(
    image: np.ndarray,
    components: int = 2,
    seed: int = 42
    ) -> np.ndarray:
    """
    Function that applies the Gaussian Mixture algorithm to a given image in order to quantize the main colours and
    segment the image accordingly.

    It returns a grayscale segmented version of the image where the regions of the different colours are line-spaced
    with values between 0 and 255.

    Parameters
    ----------
    image: ndarray
        The colour image to segment
    components: int, optional
        The number of components to find. It corresponds to the number of main region of colours that will be found
        (default: 3)
    seed: int, optional
        The seed to set for the random number generator of the algorithm (default: 42)

    Returns
    -------
    segmented_image: ndarray
        The segmented grayscale version of `image`, where the region of different colours are line-spaced with values
        between 0 and 255
    """
    # Get the main channels and reshape the image
    img_r = image.reshape(-1, 3)

    # Get the Gaussian Mixture model and fit it on the image
    gm_model = GaussianMixture(n_components=components, covariance_type='full', random_state=seed).fit(img_r)

    # Predict the main colours of the image through the model
    gmm_labels = gm_model.predict(img_r)

    # Line-space the values of the labels between 0 and 255 for clear distinction
    labels = gmm_labels.reshape(image.shape[0], image.shape[1])
    labels_map = list(np.linspace(0, 255, num=components, dtype=np.uint8))
    new_labels = np.copy(labels)
    for i, l in enumerate(np.unique(labels)):
        new_labels[labels == l] = labels_map[i]

    # Get an image segmented according to the Gaussian Mixture model
    segmented_image = new_labels.reshape(image.shape[0], image.shape[1])
    return segmented_image.astype(np.uint8)

In [8]:
images_segmented = [get_gaussian_mixture_segmented_image(images[i]) for i in range(len(images))]

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


In [1]:
# Show image
import matplotlib.pyplot as plt
plt.imshow(images_segmented[0][0], cmap='gray')

NameError: name 'images_segmented' is not defined

In [52]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(images_segmented[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [38]:
from typing import List, Tuple
from scipy.spatial.distance import cdist

def get_mean_and_inverse_covariance_matrix(
    samples: List[np.ndarray],
    ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Function that computes the mean colour and the inverse covariance colour matrix by a list of samples in a specific
    colour space and using the given channels.

    Parameters
    ----------
    samples: List[ndarray]
        The samples used to compute the mean colour and the inverse covariance colour matrix
    colour_space: ColourSpace
        The colour space the samples are turned to before computing the mean colour and inverse covariance colour matrix
        on them
    channels: Tuple[int, ...], optional
        The channels of the colour space that are used to compute the mean colour and inverse covariance colour matrix
        (default: all 3)

    Returns
    -------
    mean: ndarray
        The mean colour obtained by the samples
    inverse_covariance: ndarray
        The inverse covariance colour matrix obtained by the samples
    """
    # Get the number of channels
    channel_num = len(samples[0].shape[-1])

    # Set an array of 0s of the shape of the covariance matrix
    covariance_tot = np.zeros((channel_num, channel_num), dtype=np.float32)
    # Set an array of 0s of the shape of a mean vector of the color of the samples
    mean_tot = np.zeros((channel_num,), dtype=np.float32)

    for s in samples:
        # Turn the sample patch in the selected colour space
        s_colour_space = cv.COLOR_BGR2HSV
        # Reshape the sample patch
        s_colour_space = s_colour_space.reshape(-1, channel_num)
        # Obtain the covariance matrix and the mean for the patch
        cov, mean = cv.calcCovarMatrix(
            s_colour_space,
            None,
            cv.COVAR_NORMAL + cv.COVAR_ROWS + cv.COVAR_SCALE)
        # Add the obtained mean and the covariance to the ones of the previous patches
        covariance_tot = np.add(covariance_tot, cov)
        mean_tot = np.add(mean_tot, mean)

    # Divide the sum of means by the number of samples
    mean = mean_tot / len(samples)
    # Divide the sum of covariances by the number of samples
    covariance = covariance_tot / len(samples)

    # Invert covariance with SVD decomposition
    inv_cov = cv.invert(covariance, cv.DECOMP_SVD)[1]

    return mean, inv_cov


def get_mahalanobis_distance_segmented_image(image: np.ndarray, mean: np.ndarray,
                                             inverse_covariance_matrix: np.ndarray, threshold: float,
                                             channels: Tuple[int, ...] = (0, 1, 2)) -> np.ndarray:
    """
    Function that computes the "Mahalanobis distance" of each pixel of an image from a certain colour according to its
    mean and its inverse covariance matrix. It then sets at 0 all pixels that have a distance farther or equal to a
    certain threshold and at 255 all pixels closer than threshold. It finally returns the obtained mask.

    Parameters
    ----------
    image: ndarray
        The image in a certain colour space from which the distance mask according to the "Mahalanobis distance" is
        computed
    mean: ndarray
        The mean of the colour from which the "Mahalanobis distance" is computed
    inverse_covariance_matrix: ndarray
        The inverse covariance matrix of the colour from which the "Mahalanobis distance" is computed
    threshold: float
        Threshold used to set to 0 all pixels that have a "Mahalanobis distance" farther or equal to it and at 255 all
        pixels with a distance lower than its value.
    channels: Tuple[int, ...], optional
        The channels of the colour space that are used to compute the "Mahalanobis distance" of the image (default: all
        3)

    Returns
    -------
    mask: ndarray
        Mask of the pixels of the image with "Mahalanobis distance" lower than threshold
    """
    # Get the number of channels of the image
    channel_num = len(channels)
    # Turn the mage in the selected colour space and get the requested channels
    image = image[:, :, channels]

    # Flatten the image and change the type to `float64`
    img_flattened = image.reshape(-1, channel_num)
    img_flattened = img_flattened.astype(np.float64)

    # Get the mahalanobis distance of each pixel
    img_distances = cdist(img_flattened, mean, metric='mahalanobis', VI=inverse_covariance_matrix)

    # Reshape the flattened distance image
    img_distances = img_distances.reshape(image.shape[0], image.shape[1])

    # Obtain a distance mask where pixels more distant than `threshold` are masked
    mask = np.copy(img_distances).astype(np.uint8)
    mask[img_distances >= threshold] = 0
    mask[img_distances < threshold] = 255
    return mask

In [39]:
# images_segmented = [get_k_means_segmented_image(images[i]) for i in range(len(images))]

In [41]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(images_segmented[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…